In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
from pyspark import StorageLevel

In [2]:
conf = SparkConf().setAppName('SaveAsTable_Bucket_20240528')\
    .setMaster('yarn') \
    .set('spark.sql.shuffle.partitions', '10')\
    .set("spark.eventLog.logBlockUpdates.enabled", 'true')\
    .set("spark.sql.warehouse.dir", "hdfs:///user/ssenigov/my-spark-storage/")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)
print("spark.sql.warehouse.dir".ljust(50), spark.conf.get("spark.sql.warehouse.dir"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 07:46:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/28 07:46:11 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/05/28 07:46:11 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                             application_1711527309433_0370
spark.sql.warehouse.dir                            hdfs://adhcluster2.neoflex.ru:8020/user/ssenigov/my-spark-storage


In [3]:
import random
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-300)

names = ['Bob', 'James', 'Marek', 'Johannes', 'Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров', 'Морозов', 'Волков', 'Алексеев', 'Лебедев', 'Семенов', 'Егоров', 'Павлов', 'Козлов', 'Степанов', 'Николаев']
countries = ['rus:eu', 'bel:eu', 'usa:am', 'fra:eu', 'aus:as', 'ger:eu', 'swe:eu', 'fin:eu', 'chi:as']

In [4]:
length_customers = 1000000
customers = list()
for i in range(length_customers):
    random_date = start + (end - start) * random.random()

    if i % 3 == 0:
        country = countries[-1] # chi - 'China'
    else:
        country = countries[random.randint(0, len(countries)-2)] # without China
    
    customer = (names[random.randint(0, len(names)-1)], country, random.randint(-1000000, 1000000), random_date)
    customers.append(customer)

df_customers = spark.sparkContext.parallelize(customers, 5)\
    .toDF(schema=["name","country","balance","last_order_date"])

24/05/28 07:46:38 WARN TaskSetManager: Stage 0 contains a task of very large size (5004 KiB). The maximum recommended task size is 1000 KiB.


In [18]:
print(df_customers.rdd.getNumPartitions())
_ = spark.sql("DROP TABLE IF EXISTS tblCustomers")
_ = spark.sql("DROP TABLE IF EXISTS tblCustomers_bucketed")

5


In [19]:
df_customers.coalesce(1).write.format('csv').bucketBy(numBuckets=50, col="name")\
    .mode('overwrite').saveAsTable("tblCustomers_bucketed")

df_customers.coalesce(1).write.format('csv')\
    .mode('overwrite').saveAsTable("tblCustomers")

24/05/28 07:57:38 WARN TaskSetManager: Stage 9 contains a task of very large size (25034 KiB). The maximum recommended task size is 1000 KiB.
24/05/28 07:57:44 WARN TaskSetManager: Stage 10 contains a task of very large size (25034 KiB). The maximum recommended task size is 1000 KiB.


In [20]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

In [15]:
_ = spark.sql("DROP TABLE tblCustomers")
_ = spark.sql("DROP TABLE tblCustomers_bucketed")

In [21]:
spark.stop()